In [ ]:
!pip install carla

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.9/31.9 MB 45.2 MB/s eta 0:00:00


In [ ]:
! sudo ./CarlaUE4.sh

Refusing to run with the root privileges.


In [ ]:
# Package imports

import socket
import carla
import math
import random
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import tensorflow as tf

In [ ]:
# Creating carla & world clients
client = carla.Client('localhost', 2000)
world = client.get_world()

In [ ]:
# Selecting the control vehicle
vehicle_id = 24
vehicle = world.get_actor(vehicle_id)

In [ ]:
def process_image(image_path):
    absolute_path = os.path.abspath(image_path)
    image = Image.open(absolute_path)
    image = image.resize((800, 600))  # Resize to 800x600 pixels
    image = np.array(image) / 255.0  # Normalize to [0, 1]
    return image

In [ ]:
model = tf.keras.models.load_model('vehicle_control_model.h5')

In [ ]:
def auto_pilot(vehicle, model):
    def helper(image):
        image_file = f'./out/{image.frame}.png'
        image.save_to_disk(image_file)
        processed_image = process_image(image_file)

        # Ensure the image has the correct shape
        processed_image = np.expand_dims(processed_image, axis=0)  # Add batch dimension

        # Get predictions from the model
        predictions = model.predict(processed_image)
        throttle, steer, brake, hand_brake, reverse, manual_gear_shift, gear = predictions[0]

        # Convert predictions to the appropriate types
        hand_brake = bool(round(hand_brake))
        reverse = bool(round(reverse))
        manual_gear_shift = bool(round(manual_gear_shift))
        gear = int(round(gear))

        # Create a control object with the predicted values
        control = carla.VehicleControl(
            throttle=throttle,
            steer=steer,
            brake=brake,
            hand_brake=hand_brake,
            reverse=reverse,
            manual_gear_shift=manual_gear_shift,
            gear=gear
        )

        vehicle.apply_control(control)

    return helper

In [ ]:
# Run this listen method to turn on autopilot

camera_bp = bp_lib.find('sensor.camera.rgb')
camera_init_trans = carla.Transform(carla.Location(z=1.6, x=0.4))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

camera.listen(auto_pilot(vehicle, model))

In [ ]:
# Stop autopilot
camera.stop()